## Download tutorial files into SIMFiles folder

In [3]:
data.dir <- '/home/jupyter-user/notebooks/statgen_tutorial/SIMFiles'
out.dir <- data.dir                     # may want to write to a separate dir to avoid clutter

# Download files
urlSupport <- "https://www.mtholyoke.edu/courses/afoulkes/Data/GWAStutorial/GWASTutorial_Files.zip"
zipSupport.fn <- sprintf("%s/GWAStutorial_Files.zip", data.dir) 

# Input files
gwas.fn <- lapply(c(bed='bed',bim='bim',fam='fam',gds='gds'), function(n) sprintf("%s/GWAStutorial.%s", data.dir, n))
clinical.fn <- sprintf("%s/GWAStutorial_clinical.csv", data.dir) 
onethou.fn <- lapply(c(info='info',ped='ped'), function(n) sprintf("%s/chr16_1000g_CEU.%s", data.dir, n))
protein.coding.coords.fname <- sprintf("%s/ProCodgene_coords.csv", data.dir)

# Output files
gwaa.fname <- sprintf("%s/GWAStutorialout.txt", out.dir)
gwaa.unadj.fname <- sprintf("%s/GWAStutorialoutUnadj.txt", out.dir)
impute.out.fname <- sprintf("%s/GWAStutorial_imputationOut.csv", out.dir)
CETP.fname <- sprintf("%s/CETP_GWASout.csv", out.dir)

# Working data saved between each code snippet so each can run independently.
# Use save(data, file=working.data.fname(num))
working.data.fname <- function(num) { sprintf("%s/working.%s.Rdata", out.dir, num) }

In [8]:
# Download and unzip data needed for this tutorial
library(downloader)

download(urlSupport, zipSupport.fn)
unzip(zipSupport.fn, exdir = data.dir)

## Data preprocessing

In [9]:
library(snpStats)

# Read in PLINK files
geno <- read.plink(gwas.fn$bed, gwas.fn$bim, gwas.fn$fam, na.strings = ("-9"))

Loading required package: survival

Loading required package: Matrix



In [10]:
# Obtain the SnpMatrix object (genotypes) table from geno list
# Note: Phenotypes and covariates will be read from the clinical data file, below
genotype <- geno$genotype
print(genotype)                  # 861473 SNPs read in for 1401 subjects

A SnpMatrix with  1401 rows and  861473 columns
Row names:  10002 ... 11596 
Col names:  rs10458597 ... rs5970564 


In [11]:
#Obtain the SNP information from geno list
genoBim <- geno$map
colnames(genoBim) <- c("chr", "SNP", "gen.dist", "position", "A1", "A2")
print(head(genoBim))

           chr        SNP gen.dist position   A1 A2
rs10458597   1 rs10458597        0   564621 <NA>  C
rs12565286   1 rs12565286        0   721290    G  C
rs12082473   1 rs12082473        0   740857    T  C
rs3094315    1  rs3094315        0   752566    C  T
rs2286139    1  rs2286139        0   761732    C  T
rs11240776   1 rs11240776        0   765269    G  A


In [12]:
# Remove raw file to free up memory
rm(geno)

In [13]:
# Read in clinical file
clinical <- read.csv(clinical.fn,
                     colClasses=c("character", "factor", "factor", rep("numeric", 4)))
rownames(clinical) <- clinical$FamID
print(head(clinical))

      FamID CAD sex age  tg hdl ldl
10002 10002   1   1  60  NA  NA  NA
10004 10004   1   2  50  55  23  75
10005 10005   1   1  55 105  37  69
10007 10007   1   1  52 314  54 108
10008 10008   1   1  58 161  40  94
10009 10009   1   1  59 171  46  92


In [14]:
# Subset genotype for subject data
genotype <- genotype[clinical$FamID, ]
print(genotype)  # Tutorial: All 1401 subjects contain both clinical and genotype data

A SnpMatrix with  1401 rows and  861473 columns
Row names:  10002 ... 11596 
Col names:  rs10458597 ... rs5970564 


In [15]:
# Write genotype, genoBim, clinical for future use
save(genotype, genoBim, clinical, file = working.data.fname(1))

In [16]:
# Create SNP summary statistics (MAF, call rate, etc.)
snpsum.col <- col.summary(genotype)
print(head(snpsum.col))

           Calls Call.rate Certain.calls       RAF         MAF       P.AA
rs10458597  1398 0.9978587             1 1.0000000 0.000000000 0.00000000
rs12565286  1384 0.9878658             1 0.9483382 0.051661850 0.00433526
rs12082473  1369 0.9771592             1 0.9985391 0.001460920 0.00000000
rs3094315   1386 0.9892934             1 0.8217893 0.178210678 0.04761905
rs2286139   1364 0.9735903             1 0.8621701 0.137829912 0.02199413
rs11240776  1269 0.9057816             1 0.9988180 0.001182033 0.00000000
                  P.AB      P.BB       z.HWE
rs10458597 0.000000000 1.0000000          NA
rs12565286 0.094653179 0.9010116 -1.26529432
rs12082473 0.002921841 0.9970782  0.05413314
rs3094315  0.261183261 0.6911977 -4.03172248
rs2286139  0.231671554 0.7463343 -0.93146122
rs11240776 0.002364066 0.9976359  0.04215743


In [17]:
# Setting thresholds
call <- 0.95
minor <- 0.01

# Filter on MAF and call rate
use <- with(snpsum.col, (!is.na(MAF) & MAF > minor) & Call.rate >= call)
use[is.na(use)] <- FALSE                # Remove NA's as well

cat(ncol(genotype)-sum(use),"SNPs will be removed due to low MAF or call rate.\n") #203287 SNPs will be removed

203287 SNPs will be removed due to low MAF or call rate.


In [18]:
# Subset genotype and SNP summary data for SNPs that pass call rate and MAF criteria
genotype <- genotype[,use]
snpsum.col <- snpsum.col[use,]

print(genotype)                           # 658186 SNPs remain

A SnpMatrix with  1401 rows and  658186 columns
Row names:  10002 ... 11596 
Col names:  rs12565286 ... rs5970564 


In [19]:
# Write subsetted genotype data and derived results for future use
save(genotype, snpsum.col, genoBim, clinical, file=working.data.fname(2))

In [21]:
# Sample level filtering

#source("globals.R")

# load data created in previous snippets
load(working.data.fname(2))

library(snpStats)

In [22]:
library(SNPRelate)                      # LD pruning, relatedness, PCA
library(plyr)

# Create sample statistics (Call rate, Heterozygosity)
snpsum.row <- row.summary(genotype)

# Add the F stat (inbreeding coefficient) to snpsum.row
MAF <- snpsum.col$MAF
callmatrix <- !is.na(genotype)
hetExp <- callmatrix %*% (2*MAF*(1-MAF))
hetObs <- with(snpsum.row, Heterozygosity*(ncol(genotype))*Call.rate)
snpsum.row$hetF <- 1-(hetObs/hetExp)

head(snpsum.row)

Loading required package: gdsfmt

SNPRelate -- supported by Streaming SIMD Extensions 2 (SSE2)



ERROR: Error: cannot allocate vector of size 6.9 Gb


In [ ]:
# Setting thresholds
sampcall <- 0.95    # Sample call rate cut-off
hetcutoff <- 0.1    # Inbreeding coefficient cut-off

sampleuse <- with(snpsum.row, !is.na(Call.rate) & Call.rate > sampcall & abs(hetF) <= hetcutoff)
sampleuse[is.na(sampleuse)] <- FALSE    # remove NA's as well
cat(nrow(genotype)-sum(sampleuse), "subjects will be removed due to low sample call rate or inbreeding coefficient.\n") #0 subjects removed

In [ ]:
# Subset genotype and clinical data for subjects who pass call rate and heterozygosity crtieria
genotype <- genotype[sampleuse,]
clinical<- clinical[ rownames(genotype), ]